In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch import Tensor
import torch.nn.functional as F
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter

In [2]:
writer = SummaryWriter()

In [3]:
train_set = pd.read_csv("data/final_format/train_set.csv",header=None).to_numpy()
train_label = pd.read_csv("data/final_format/train_label.csv",header=None).to_numpy()
test_set = pd.read_csv("data/final_format/test_set.csv",header=None).to_numpy()
test_label = pd.read_csv("data/final_format/test_label.csv",header=None).to_numpy()

In [4]:
print(train_set.shape, train_label.shape, test_set.shape, test_label.shape)

(14393, 4096) (14393, 1) (3599, 4096) (3599, 1)


In [5]:
#delet first row data
train_set = train_set[1:]
train_label = train_label[1:]
test_set = test_set[1:]
test_label = test_label[1:]
print(train_set.shape, train_label.shape, test_set.shape, test_label.shape)

(14392, 4096) (14392, 1) (3598, 4096) (3598, 1)


In [6]:
train_set = train_set.reshape((-1,1,64,64))
test_set = test_set.reshape((-1,1,64,64))
print(train_set.shape, train_label.shape, test_set.shape, test_label.shape)

(14392, 1, 64, 64) (14392, 1) (3598, 1, 64, 64) (3598, 1)


In [7]:
train_label = train_label.reshape(-1)
test_label = test_label.reshape(-1)

print(train_set.shape, train_label.shape, test_set.shape, test_label.shape)

(14392, 1, 64, 64) (14392,) (3598, 1, 64, 64) (3598,)


In [8]:
# Hyper parameters
num_epochs = 300
num_classes = 4
batch_size = 64
learning_rate = 1e-3

In [9]:
train_set_tensor = Tensor(train_set) 
train_label_tensor = Tensor(train_label).type(torch.LongTensor)

train_dataset = TensorDataset(train_set_tensor,train_label_tensor) 
train_loader = DataLoader(train_dataset, batch_size=batch_size) 

test_set_tensor = Tensor(test_set) 
test_label_tensor = Tensor(test_label).type(torch.LongTensor)

test_dataset = TensorDataset(test_set_tensor,test_label_tensor) 
test_loader = DataLoader(test_dataset, batch_size=batch_size) 

In [10]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [11]:
# Convolutional neural network (two convolutional layers)
class ThinResCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(ThinResCNN, self).__init__()
        self.c1 = nn.Conv2d(1, 48, kernel_size=1, padding='same')
        self.c2 = nn.Conv2d(48, 48, kernel_size=3,padding='same')
        self.c3 = nn.Conv2d(48, 96, kernel_size=1,padding='same') 
        self.c4 = nn.Conv2d(97, 96, kernel_size=1, padding='same')
        self.c5 = nn.Conv2d(96, 96, kernel_size=3,padding='same')
        self.c6 = nn.Conv2d(96, 128, kernel_size=1, padding='same')
        self.c7 = nn.Conv2d(225, 128, kernel_size=3, padding='same')
        self.c8 = nn.Conv2d(128, 128, kernel_size=3,padding='same')
        self.c9 = nn.Conv2d(128, 256, kernel_size=3, padding='same')
        
        self.d1 = nn.Dropout(p=0.25)
        self.d2 = nn.Dropout(p=0.25)
        self.d3 = nn.Dropout(p=0.25)
        self.d4 = nn.Dropout(p=0.25)
        self.d5 = nn.Dropout(p=0.25)
        self.d6 = nn.Dropout(p=0.25) 
        self.d7 = nn.Dropout(p=0.25)
        self.d8 = nn.Dropout(p=0.25)
        self.d9 = nn.Dropout(p=0.25) 
        self.d10 = nn.Dropout(p=0.25) 
        
        self.bn1 =  nn.BatchNorm2d(1)
        self.bn2 =  nn.BatchNorm2d(48)
        self.bn3 =  nn.BatchNorm2d(48)
        self.bn4 =  nn.BatchNorm2d(97)
        self.bn5 =  nn.BatchNorm2d(96)
        self.bn6 =  nn.BatchNorm2d(96)
        self.bn7 =  nn.BatchNorm2d(225)
        self.bn8 =  nn.BatchNorm2d(128)
        self.bn9 =  nn.BatchNorm2d(128)
        self.bn10 = nn.BatchNorm1d(512)
        
        self.fc1 = nn.Linear(481*8*8, 512)
        self.fc2 = nn.Linear(512, num_classes)
        
        
    def forward(self, x):
        C1 = self.c1(self.bn1(x))
        C1 = F.leaky_relu(C1,0.2)
        C1 = self.d1(C1)
        C2 = self.c2(self.bn2(C1))
        C2 = F.leaky_relu(C2,0.2)
        C2 = self.d2(C2)
        C3 = self.c3(self.bn3(C2))
        sum1 = torch.cat((x, C3), dim=1)
        sum1 = F.leaky_relu(sum1,0.2)
        sum1 = self.d3(sum1)
        M1 = F.max_pool2d(sum1, kernel_size=2, stride=2)
         
        C4 = self.c4(self.bn4(M1))
        C4 = F.leaky_relu(C4,0.2)
        C4 = self.d4(C4)
        C5 = self.c5(self.bn5(C4))
        C5 = F.leaky_relu(C5,0.2)
        C5 = self.d5(C5)
        C6 = self.c6(self.bn6(C5))
        sum2 = torch.cat((M1, C6), dim=1)
        sum2 = F.leaky_relu(sum2,0.2)
        sum2 = self.d6(sum2)
        M2 = F.max_pool2d(sum2, kernel_size=2, stride=2)
        
        C7 = self.c7(self.bn7(M2))
        C7 = F.leaky_relu(C7,0.2)
        C7 = self.d7(C7)
        C8 = self.c8(self.bn8(C7))
        C8 = F.leaky_relu(C8,0.2)
        C8 = self.d8(C8)
        C9 = self.c9(self.bn9(C8))
        sum3 = torch.cat((M2, C9), dim=1)
        
        
        sum3 = F.leaky_relu(sum3,0.2)
        sum3 = self.d9(sum3)
        M3 = F.max_pool2d(sum3, kernel_size=2, stride=2)
        

        F1 = M3.reshape(M3.size(0), -1)
        Fc1 = self.fc1(F1)
        Fc1 = self.bn10(Fc1)
        Fc1 = F.leaky_relu(Fc1,0.2)
        Fc1 = self.d10(Fc1)
        Fc2 = self.fc2(Fc1)
       
        return Fc2
    


In [12]:
model = ThinResCNN(num_classes).to(device)

In [13]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-3) 
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
milestones = [50,100,150,200,250]
milestones = [a * len(train_loader) for a in milestones]
scheduler = MultiStepLR(optimizer, milestones=milestones, gamma=0.5)

In [ ]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    correct=0
    total=0
    running_loss = 0
    for i, (X, Y) in enumerate(train_loader):
        X = X.to(device)
        Y = Y.to(device)
        
        
        # Forward pass
        outputs = model(X)
        loss = criterion(outputs, Y)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        #scheduler.step() 
        #print(scheduler.get_last_lr()[0])
      
        optimizer.step()
        scheduler.step() 
        #print(optimizer.param_groups[0]["lr"])
        
        _, predicted = outputs.max(1)
        total += Y.size(0)
        correct += predicted.eq(Y).sum().item()
        running_loss += loss.item()
        accu=100.*correct/total
        train_loss = running_loss/(i+1)
        print ('Epoch [{}/{}], Step [{}/{}], Training Accuracy: {:.4f}%, Training Loss: {:.4f}%'.format(epoch+1, num_epochs, i+1, total_step, accu, train_loss))
    
   
        #writer.add_scalar(f'train/accuracy', accu, epoch)
        #writer.add_scalar(f'train/loss', train_loss, epoch)
        writer.add_scalars(f'train/accuracy_loss', {
            'accuracy': accu,
            'loss': train_loss,
        }, epoch)
        
        

Epoch [1/300], Step [1/225], Training Accuracy: 26.5625%, Training Loss: 1.4292%
Epoch [1/300], Step [2/225], Training Accuracy: 23.4375%, Training Loss: 1.9829%
Epoch [1/300], Step [3/225], Training Accuracy: 28.1250%, Training Loss: 1.8179%
Epoch [1/300], Step [4/225], Training Accuracy: 28.1250%, Training Loss: 1.7856%
Epoch [1/300], Step [5/225], Training Accuracy: 27.8125%, Training Loss: 1.7718%
Epoch [1/300], Step [6/225], Training Accuracy: 27.3438%, Training Loss: 1.7672%
Epoch [1/300], Step [7/225], Training Accuracy: 27.0089%, Training Loss: 1.7211%
Epoch [1/300], Step [8/225], Training Accuracy: 26.7578%, Training Loss: 1.6988%
Epoch [1/300], Step [9/225], Training Accuracy: 27.4306%, Training Loss: 1.6657%
Epoch [1/300], Step [10/225], Training Accuracy: 27.3438%, Training Loss: 1.6430%
Epoch [1/300], Step [11/225], Training Accuracy: 27.1307%, Training Loss: 1.6326%
Epoch [1/300], Step [12/225], Training Accuracy: 27.7344%, Training Loss: 1.6104%
Epoch [1/300], Step [13/2

Epoch [1/300], Step [104/225], Training Accuracy: 30.9195%, Training Loss: 1.4046%
Epoch [1/300], Step [105/225], Training Accuracy: 30.9077%, Training Loss: 1.4046%
Epoch [1/300], Step [106/225], Training Accuracy: 30.9552%, Training Loss: 1.4040%
Epoch [1/300], Step [107/225], Training Accuracy: 31.0164%, Training Loss: 1.4036%
Epoch [1/300], Step [108/225], Training Accuracy: 31.0619%, Training Loss: 1.4035%
Epoch [1/300], Step [109/225], Training Accuracy: 31.0206%, Training Loss: 1.4032%
Epoch [1/300], Step [110/225], Training Accuracy: 31.0227%, Training Loss: 1.4025%
Epoch [1/300], Step [111/225], Training Accuracy: 30.9685%, Training Loss: 1.4022%
Epoch [1/300], Step [112/225], Training Accuracy: 31.0268%, Training Loss: 1.4022%
Epoch [1/300], Step [113/225], Training Accuracy: 30.9735%, Training Loss: 1.4022%
Epoch [1/300], Step [114/225], Training Accuracy: 31.0444%, Training Loss: 1.4008%
Epoch [1/300], Step [115/225], Training Accuracy: 31.1141%, Training Loss: 1.4005%
Epoc

Epoch [1/300], Step [206/225], Training Accuracy: 32.7518%, Training Loss: 1.3735%
Epoch [1/300], Step [207/225], Training Accuracy: 32.7446%, Training Loss: 1.3736%
Epoch [1/300], Step [208/225], Training Accuracy: 32.8050%, Training Loss: 1.3732%
Epoch [1/300], Step [209/225], Training Accuracy: 32.7975%, Training Loss: 1.3731%
Epoch [1/300], Step [210/225], Training Accuracy: 32.7604%, Training Loss: 1.3732%
Epoch [1/300], Step [211/225], Training Accuracy: 32.7829%, Training Loss: 1.3729%
Epoch [1/300], Step [212/225], Training Accuracy: 32.7904%, Training Loss: 1.3725%
Epoch [1/300], Step [213/225], Training Accuracy: 32.7905%, Training Loss: 1.3723%
Epoch [1/300], Step [214/225], Training Accuracy: 32.8052%, Training Loss: 1.3721%
Epoch [1/300], Step [215/225], Training Accuracy: 32.8343%, Training Loss: 1.3719%
Epoch [1/300], Step [216/225], Training Accuracy: 32.8125%, Training Loss: 1.3717%
Epoch [1/300], Step [217/225], Training Accuracy: 32.8485%, Training Loss: 1.3710%
Epoc

Epoch [2/300], Step [85/225], Training Accuracy: 36.1581%, Training Loss: 1.3103%
Epoch [2/300], Step [86/225], Training Accuracy: 36.1010%, Training Loss: 1.3096%
Epoch [2/300], Step [87/225], Training Accuracy: 36.0812%, Training Loss: 1.3104%
Epoch [2/300], Step [88/225], Training Accuracy: 36.1328%, Training Loss: 1.3100%
Epoch [2/300], Step [89/225], Training Accuracy: 36.0604%, Training Loss: 1.3116%
Epoch [2/300], Step [90/225], Training Accuracy: 36.0590%, Training Loss: 1.3121%
Epoch [2/300], Step [91/225], Training Accuracy: 36.0405%, Training Loss: 1.3121%
Epoch [2/300], Step [92/225], Training Accuracy: 36.0054%, Training Loss: 1.3110%
Epoch [2/300], Step [93/225], Training Accuracy: 35.9375%, Training Loss: 1.3119%
Epoch [2/300], Step [94/225], Training Accuracy: 35.9874%, Training Loss: 1.3107%
Epoch [2/300], Step [95/225], Training Accuracy: 35.9211%, Training Loss: 1.3118%
Epoch [2/300], Step [96/225], Training Accuracy: 35.9701%, Training Loss: 1.3111%
Epoch [2/300], S

Epoch [2/300], Step [184/225], Training Accuracy: 36.9650%, Training Loss: 1.2934%
Epoch [2/300], Step [185/225], Training Accuracy: 37.0186%, Training Loss: 1.2926%
Epoch [2/300], Step [186/225], Training Accuracy: 37.0548%, Training Loss: 1.2924%
Epoch [2/300], Step [187/225], Training Accuracy: 37.0822%, Training Loss: 1.2922%
Epoch [2/300], Step [188/225], Training Accuracy: 37.1094%, Training Loss: 1.2919%
Epoch [2/300], Step [189/225], Training Accuracy: 37.1114%, Training Loss: 1.2914%
Epoch [2/300], Step [190/225], Training Accuracy: 37.1217%, Training Loss: 1.2917%
Epoch [2/300], Step [191/225], Training Accuracy: 37.1237%, Training Loss: 1.2913%
Epoch [2/300], Step [192/225], Training Accuracy: 37.1826%, Training Loss: 1.2910%
Epoch [2/300], Step [193/225], Training Accuracy: 37.1924%, Training Loss: 1.2909%
Epoch [2/300], Step [194/225], Training Accuracy: 37.2020%, Training Loss: 1.2905%
Epoch [2/300], Step [195/225], Training Accuracy: 37.2196%, Training Loss: 1.2904%
Epoc

...

Epoch [8/300], Step [53/225], Training Accuracy: 54.4517%, Training Loss: 0.9263%
Epoch [8/300], Step [54/225], Training Accuracy: 54.4271%, Training Loss: 0.9274%
Epoch [8/300], Step [55/225], Training Accuracy: 54.4886%, Training Loss: 0.9270%
Epoch [8/300], Step [56/225], Training Accuracy: 54.5480%, Training Loss: 0.9257%
Epoch [8/300], Step [57/225], Training Accuracy: 54.6875%, Training Loss: 0.9227%
Epoch [8/300], Step [58/225], Training Accuracy: 54.7953%, Training Loss: 0.9229%
Epoch [8/300], Step [59/225], Training Accuracy: 54.8464%, Training Loss: 0.9210%
Epoch [8/300], Step [60/225], Training Accuracy: 54.8958%, Training Loss: 0.9198%
Epoch [8/300], Step [61/225], Training Accuracy: 54.9180%, Training Loss: 0.9189%
Epoch [8/300], Step [62/225], Training Accuracy: 54.9143%, Training Loss: 0.9181%
Epoch [8/300], Step [63/225], Training Accuracy: 54.8363%, Training Loss: 0.9195%
Epoch [8/300], Step [64/225], Training Accuracy: 54.8828%, Training Loss: 0.9207%
Epoch [8/300], S

Epoch [8/300], Step [153/225], Training Accuracy: 54.3505%, Training Loss: 0.9292%
Epoch [8/300], Step [154/225], Training Accuracy: 54.3527%, Training Loss: 0.9288%
Epoch [8/300], Step [155/225], Training Accuracy: 54.3750%, Training Loss: 0.9287%
Epoch [8/300], Step [156/225], Training Accuracy: 54.4371%, Training Loss: 0.9282%
Epoch [8/300], Step [157/225], Training Accuracy: 54.4188%, Training Loss: 0.9282%
Epoch [8/300], Step [158/225], Training Accuracy: 54.3809%, Training Loss: 0.9294%
Epoch [8/300], Step [159/225], Training Accuracy: 54.3239%, Training Loss: 0.9300%
Epoch [8/300], Step [160/225], Training Accuracy: 54.3164%, Training Loss: 0.9299%
Epoch [8/300], Step [161/225], Training Accuracy: 54.3769%, Training Loss: 0.9296%
Epoch [8/300], Step [162/225], Training Accuracy: 54.4560%, Training Loss: 0.9290%
Epoch [8/300], Step [163/225], Training Accuracy: 54.4383%, Training Loss: 0.9290%
Epoch [8/300], Step [164/225], Training Accuracy: 54.4588%, Training Loss: 0.9283%
Epoc

Epoch [9/300], Step [32/225], Training Accuracy: 55.1270%, Training Loss: 0.9140%
Epoch [9/300], Step [33/225], Training Accuracy: 55.1136%, Training Loss: 0.9115%
Epoch [9/300], Step [34/225], Training Accuracy: 54.8254%, Training Loss: 0.9137%
Epoch [9/300], Step [35/225], Training Accuracy: 54.8214%, Training Loss: 0.9162%
Epoch [9/300], Step [36/225], Training Accuracy: 54.7309%, Training Loss: 0.9168%
Epoch [9/300], Step [37/225], Training Accuracy: 54.9831%, Training Loss: 0.9152%
Epoch [9/300], Step [38/225], Training Accuracy: 54.8520%, Training Loss: 0.9144%
Epoch [9/300], Step [39/225], Training Accuracy: 54.8077%, Training Loss: 0.9152%
Epoch [9/300], Step [40/225], Training Accuracy: 54.7266%, Training Loss: 0.9174%
Epoch [9/300], Step [41/225], Training Accuracy: 54.4588%, Training Loss: 0.9203%
Epoch [9/300], Step [42/225], Training Accuracy: 54.5015%, Training Loss: 0.9200%
Epoch [9/300], Step [43/225], Training Accuracy: 54.4331%, Training Loss: 0.9201%
Epoch [9/300], S

Epoch [9/300], Step [132/225], Training Accuracy: 54.5099%, Training Loss: 0.9155%
Epoch [9/300], Step [133/225], Training Accuracy: 54.4878%, Training Loss: 0.9161%
Epoch [9/300], Step [134/225], Training Accuracy: 54.4776%, Training Loss: 0.9168%
Epoch [9/300], Step [135/225], Training Accuracy: 54.5255%, Training Loss: 0.9163%
Epoch [9/300], Step [136/225], Training Accuracy: 54.5267%, Training Loss: 0.9160%
Epoch [9/300], Step [137/225], Training Accuracy: 54.5278%, Training Loss: 0.9161%
Epoch [9/300], Step [138/225], Training Accuracy: 54.5516%, Training Loss: 0.9153%
Epoch [9/300], Step [139/225], Training Accuracy: 54.5301%, Training Loss: 0.9155%
Epoch [9/300], Step [140/225], Training Accuracy: 54.5424%, Training Loss: 0.9150%
Epoch [9/300], Step [141/225], Training Accuracy: 54.5878%, Training Loss: 0.9145%
Epoch [9/300], Step [142/225], Training Accuracy: 54.6215%, Training Loss: 0.9142%
Epoch [9/300], Step [143/225], Training Accuracy: 54.6329%, Training Loss: 0.9138%
Epoc

Epoch [10/300], Step [6/225], Training Accuracy: 59.1146%, Training Loss: 0.8890%
Epoch [10/300], Step [7/225], Training Accuracy: 58.2589%, Training Loss: 0.8872%
Epoch [10/300], Step [8/225], Training Accuracy: 58.3984%, Training Loss: 0.8911%
Epoch [10/300], Step [9/225], Training Accuracy: 58.5069%, Training Loss: 0.8945%
Epoch [10/300], Step [10/225], Training Accuracy: 57.9688%, Training Loss: 0.9118%
Epoch [10/300], Step [11/225], Training Accuracy: 57.9545%, Training Loss: 0.9093%
Epoch [10/300], Step [12/225], Training Accuracy: 58.0729%, Training Loss: 0.9047%
Epoch [10/300], Step [13/225], Training Accuracy: 59.1346%, Training Loss: 0.8939%
Epoch [10/300], Step [14/225], Training Accuracy: 59.3750%, Training Loss: 0.8886%
Epoch [10/300], Step [15/225], Training Accuracy: 59.3750%, Training Loss: 0.8895%
Epoch [10/300], Step [16/225], Training Accuracy: 59.0820%, Training Loss: 0.8879%
Epoch [10/300], Step [17/225], Training Accuracy: 59.0074%, Training Loss: 0.8822%
Epoch [1

In [ ]:
# Test the model

model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for X, Y in test_loader:
        X = X.to(device)
        Y = Y.to(device)
        outputs = model(X)
        _, predicted = torch.max(outputs.data, 1)
        total += Y.size(0)
        correct += (predicted == Y).sum().item()

    print('Test Accuracy : {} %'.format(100 * correct / total))

# Save the model checkpoint
#torch.save(model.state_dict(), 'model.ckpt')